In [0]:
%sh
rm -r /dbfs/delta_lab
mkdir /dbfs/delta_lab
wget -O /dbfs/delta_lab/products.csv https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/products.csv

--2024-03-25 09:51:27--  https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/products.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14372 (14K) [text/plain]
Saving to: ‘/dbfs/delta_lab/products.csv’

     0K .......... ....                                       100% 3.58M=0.004s

2024-03-25 09:51:27 (3.58 MB/s) - ‘/dbfs/delta_lab/products.csv’ saved [14372/14372]



In [0]:
df = spark.read.load('/delta_lab/products.csv', format='csv',
header=True)
display(df.limit(10))

ProductID,ProductName,Category,ListPrice
771,"Mountain-100 Silver, 38",Mountain Bikes,3399.9900
772,"Mountain-100 Silver, 42",Mountain Bikes,3399.9900
773,"Mountain-100 Silver, 44",Mountain Bikes,3399.9900
774,"Mountain-100 Silver, 48",Mountain Bikes,3399.9900
775,"Mountain-100 Black, 38",Mountain Bikes,3374.9900
776,"Mountain-100 Black, 42",Mountain Bikes,3374.9900
777,"Mountain-100 Black, 44",Mountain Bikes,3374.9900
778,"Mountain-100 Black, 48",Mountain Bikes,3374.9900
779,"Mountain-200 Silver, 38",Mountain Bikes,2319.9900
780,"Mountain-200 Silver, 42",Mountain Bikes,2319.9900


In [0]:
delta_table_path = "/delta/products-delta"
df.write.format("delta").save(delta_table_path)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1909370656367984>, line 2
      1 delta_table_path = "/delta/products-delta"
----> 2 df.write.format("delta").save(delta_table_path)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1463, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
   1461     self._jwrite.save()
   1462 else:
-> 1463     self._jwrite.save(path)

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py:1322, in JavaMember.__call__(se

In [0]:
%sh
ls /dbfs/delta/products-delta

_delta_log
part-00000-a99d29cd-d8aa-4735-ba2b-794e202ee40e.c000.snappy.parquet


In [0]:
from delta.tables import *
from pyspark.sql.functions import *

# Create a deltaTable object
deltaTable = DeltaTable.forPath(spark, delta_table_path)
# Update the table (reduce price of product 771 by 10%)
deltaTable.update(
 condition = "ProductID == 771",
 set = { "ListPrice": "ListPrice * 0.9" })
# View the updated data as a dataframe
deltaTable.toDF().show(10)

+---------+--------------------+--------------+----------+
|ProductID|         ProductName|      Category| ListPrice|
+---------+--------------------+--------------+----------+
|      771|Mountain-100 Silv...|Mountain Bikes|2478.59271|
|      772|Mountain-100 Silv...|Mountain Bikes| 3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes| 3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes| 3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes| 3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes| 3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes| 3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes| 3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes| 2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes| 2319.9900|
+---------+--------------------+--------------+----------+
only showing top 10 rows



In [0]:
new_df = spark.read.format("delta").load(delta_table_path)
new_df.show(10)

+---------+--------------------+--------------+----------+
|ProductID|         ProductName|      Category| ListPrice|
+---------+--------------------+--------------+----------+
|      771|Mountain-100 Silv...|Mountain Bikes|2478.59271|
|      772|Mountain-100 Silv...|Mountain Bikes| 3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes| 3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes| 3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes| 3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes| 3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes| 3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes| 3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes| 2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes| 2319.9900|
+---------+--------------------+--------------+----------+
only showing top 10 rows



#Explora la tabla y los viajes en el tiempo

In [0]:
new_df = spark.read.format("delta").option("versionAsOf",
0).load(delta_table_path)
new_df.show(10)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [0]:
deltaTable.history(10).show(10, False, True)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 2                                                                                                                                                                                                                                                                                               
 timestamp           | 2024-03-25 09:54:04                                                                                                                                                                                                                                                                             
 userId              | 623293643459307                          

#Crear tablas de catálogo

In [0]:
spark.sql("CREATE DATABASE AdventureWorks")
spark.sql("CREATE TABLE AdventureWorks.ProductsExternal USING DELTA LOCATION '{0}'".format(delta_table_path))
spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsExternal").show(truncate=False)

DataFrame[]

In [0]:
delta_table_path2 = "dbfs:/delta/products-delta"
spark.sql("CREATE TABLE AdventureWorks.ProductsExternalAsier USING DELTA LOCATION '{0}'".format(delta_table_path2))

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1909370656368007>, line 2
      1 delta_table_path2 = "abfss:/delta/products-delta"
----> 2 spark.sql("CREATE TABLE AdventureWorks.ProductsExternalAsier USING DELTA LOCATION '{0}'".format(delta_table_path2))

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1602, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1598         assert self._jvm is not None
   1599         litArgs = self._jvm.PythonUtils.toArray(
   1600             [_to_java_column(li

In [0]:
%sql
USE adventureworks;
SHOW TABLES;

database,tableName,isTemporary
adventureworks,productsexternalasier,false
adventureworks,productsexternalasier2,false


In [0]:
new_df.write.format("delta").saveAsTable("AdventureWorks.ProductsManaged")
spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsManaged").show(truncate=False)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1909370656368064>, line 1
----> 1 new_df.write.format("delta").saveAsTable("AdventureWorks.ProductsManaged")
      2 spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsManaged").show(truncate=False)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1586, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
   1584 if format is not None:
   1585     self.format(format)
-> 1586 self._jwrite.saveAsTable(name)

File /databric

In [0]:
%sql
USE AdventureWorks;
SELECT * FROM ProductsManaged;

ProductID,ProductName,Category,ListPrice
771,"Mountain-100 Silver, 38",Mountain Bikes,3399.9900
772,"Mountain-100 Silver, 42",Mountain Bikes,3399.9900
773,"Mountain-100 Silver, 44",Mountain Bikes,3399.9900
774,"Mountain-100 Silver, 48",Mountain Bikes,3399.9900
775,"Mountain-100 Black, 38",Mountain Bikes,3374.9900
776,"Mountain-100 Black, 42",Mountain Bikes,3374.9900
777,"Mountain-100 Black, 44",Mountain Bikes,3374.9900
778,"Mountain-100 Black, 48",Mountain Bikes,3374.9900
779,"Mountain-200 Silver, 38",Mountain Bikes,2319.9900
780,"Mountain-200 Silver, 42",Mountain Bikes,2319.9900


In [0]:
%sql
USE AdventureWorks;
SHOW TABLES;


database,tableName,isTemporary
adventureworks,productsexternalasier,false
adventureworks,productsexternalasier2,false
adventureworks,productsexternalguille,false
adventureworks,productsmanaged,false


In [0]:
%sh
echo "External table:"
ls /dbfs/delta/products-delta
echo
echo "Managed table:"
ls /dbfs/user/hive/warehouse/adventureworks.db/productsmanaged

External table:
_delta_log
part-00000-07ac446f-5653-4556-95b8-651a17aa5a6d.c000.snappy.parquet
part-00000-1ce2ed77-ec62-45b2-8f15-57f3587c51aa.c000.snappy.parquet
part-00000-6757aa78-8599-46a2-8700-1004d3f3cc0b.c000.snappy.parquet
part-00000-6c07fec1-1fbd-47f4-a9ae-65f7b6c5a63a.c000.snappy.parquet
part-00000-745c7791-5b64-44a2-acb3-ea3d10ba756b.c000.snappy.parquet
part-00000-76af58f9-326e-4957-ba0f-6b66c6f415cb.c000.snappy.parquet
part-00000-a99d29cd-d8aa-4735-ba2b-794e202ee40e.c000.snappy.parquet

Managed table:


ls: cannot access '/dbfs/user/hive/warehouse/adventureworks.db/productsmanaged': No such file or directory


#Utilice tablas delta para transmitir datos

In [0]:
%sh 
rm -r /dbfs/device_stream 
mkdir /dbfs/device_stream 
wget -O /dbfs/device_stream/devices1.json https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/devices1.json

rm: cannot remove '/dbfs/device_stream': No such file or directory
--2024-03-25 10:34:14--  https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/devices1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 296 [text/plain]
Saving to: ‘/dbfs/device_stream/devices1.json’

     0K                                                       100% 10.3M=0s

2024-03-25 10:34:14 (10.3 MB/s) - ‘/dbfs/device_stream/devices1.json’ saved [296/296]



In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Create a stream that reads data from the folder, using a JSON schema
inputPath = '/device_stream/'
jsonSchema = StructType([
StructField("device", StringType(), False),
StructField("status", StringType(), False)
])
iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger",1).json(inputPath)
print("Source stream created...")

Source stream created...


In [0]:
# Write the stream to a delta table
delta_stream_table_path = '/delta/iotdevicedata'
checkpointpath = '/delta/checkpoint'
deltastream = iotstream.writeStream.format("delta").option("checkpointLocation",checkpointpath).start(delta_stream_table_path)
print("Streaming to delta sink...")


Streaming to delta sink...


In [0]:
# Read the data in delta format into a dataframe
df = spark.read.format("delta").load(delta_stream_table_path)
display(df)

device,status
Dev1,ok
Dev1,ok
Dev1,ok
Dev2,error
Dev1,ok
Dev1,error
Dev2,ok
Dev2,error
Dev1,ok


In [0]:
print(delta_stream_table_path)

/delta/iotdevicedata


In [0]:
# Assuming you are using Azure Blob Storage
delta_stream_table_path = "abfss://unity-catalog-storage@dbstoragehze6mq7ttr4r6.dfs.core.windows.net/delta/iotdevicedata"

# Create a catalog table based on the streaming sink
spark.sql("CREATE TABLE IotDeviceData USING DELTA LOCATION '{0}'".format(delta_stream_table_path))


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1909370656368109>, line 5
      2 delta_stream_table_path = "abfss://unity-catalog-storage@dbstoragehze6mq7ttr4r6.dfs.core.windows.net/delta/iotdevicedata"
      4 # Create a catalog table based on the streaming sink
----> 5 spark.sql("CREATE TABLE IotDeviceData USING DELTA LOCATION '{0}'".format(delta_stream_table_path))

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1602, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1598         assert se